In [ ]:
# 1. import dependencies
#%pip install stable-baselines3[extra]

In [1]:
import os
import gymnasium as gym
#import time
from stable_baselines3 import PPO  #first algorithm import
from stable_baselines3.common.vec_env import DummyVecEnv  #wrapper around our environment
from stable_baselines3.common.evaluation import evaluate_policy  
#to test our reward and standard deviation  

In [2]:
# 2. LOAD ENVIRONMENT
#preinstalled open ai gym env
environment_name = 'CartPole-v1'
env = gym.make(environment_name)  # making our own env


In [ ]:
# loop to test environment
episodes = 5  # number of episodes, loop 5 times
for episode in range(1, episodes+1):  # loop to test environment
    state = env.reset()  # get initial set of observation . random seed
    done = False  
    score = 0

    while not done:
        env.render()  #view the environment
        #time.sleep(0.1)
        action = env.action_space.sample()  #genertating random action 0 or 1
        n_state, reward, truncated, done, info = env.step(action)  #pass through our action in the environment
        score += reward  #accumulating the reward
    print('Episode:{} Score:{}' .format(episode, score))  
env.close()

In [4]:
env.reset()

(array([-0.01024487, -0.01341263,  0.03435903, -0.03483175], dtype=float32),
 {})

In [14]:
env.action_space

Discrete(2)

In [ ]:
#UNDERSTANDING ENVIRONMENT

env.action_space
# action 0 or 1. 0 will push to left and 1 will push to right
# type of action space


In [ ]:
env.observation_space


In [ ]:
env.observation_space.sample()
# type of observation space
# car_position, car_velocity, pole_angle, pole_velocity

In [ ]:
# make directories first
log_path = os.path.join('Training', 'Logs')

In [ ]:
log_path

In [ ]:
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)


In [ ]:
env.close()

In [ ]:
# SAVE AND RELOAD MODEL
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [ ]:
model.save(PPO_Path)
model = PPO.load(PPO_Path, env=env)

In [ ]:
# EVALUATE MODEL
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# render is for visualization purposes so if we dont wanna see then render false

In [ ]:
#action, _ = model.predict(obs)

In [ ]:
# TESTING MODEL
episodes = 5  # number of episodes, loop 5 times
for episode in range(1, episodes+1):  # loop to test environment
    obs = env.reset()  # get initial set of observation
    done = False  
    score = 0

    while not done:
        env.render()  #view the environment
        #time.sleep(0.1)
        action, _ = model.predict(obs)  #using model here
        obs_state, reward, done, info = env.step(action)  #pass through our action in the environment
        score += reward  #accumulating the reward
    print('Episode:{} Score:{}' .format(episode, score))  
env.close()

In [ ]:
obs = env.reset()

In [ ]:
action, _ = model.predict(obs)

In [ ]:
env.action_space.sample()

In [ ]:
env.step(action)

In [ ]:
# VIEWING LOGS IN TENSORBOARD

training_log_path = os.path.join(log_path, 'PPO_2')

In [ ]:
training_log_path

In [ ]:
#!tensorboard --logdir={training_log_path}
# view everything in tensorboard, do in terminal not here

In [ ]:
# ADDING CALLBACK TO OUR TRAINING STAGE
# Dependency is a state in which one object uses a function of another object. 
# Data dependency, which describes a dependence relation between statements in a program.
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('Training', 'Saved Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
# CHANGING POLICIES
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)